solution for issue3_trans_vary.py
(Aziz - June 20,2022, 5;20 am, to bnl emails)

 specific pass call Optimize1qGatesDecomposition which make possible to reduce ZXZXZ to U2 - if possible.
 
To avoid this and use only ZXZXZ decomposition, you can use the optimization level 0(but the circuit layout will not be noise optimized) or
you can create the level 3 pass manager manually and remove the Optimize1qGatesDecomposition pass, I attached a notebook for this.
 

In [1]:
#!/usr/bin/env python3

from qiskit.test.mock import FakeLima
from qiskit.providers.aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import level_3_pass_manager
from qiskit.transpiler import PassManagerConfig
from qiskit.transpiler import CouplingMap
import qiskit as qk
from pprint import pprint

device_backend = FakeLima()
backend=AerSimulator.from_backend(device_backend)
seed=111
print('backend=',backend,seed)


/tmp/ipykernel_602/2692642352.py:3: DeprecationWarning: The module 'qiskit.test.mock' is deprecated since Qiskit Terra 0.21.0, and will be removed 3 months or more later. Instead, you should import the desired object directly 'qiskit.providers.fake_provider'.
  from qiskit.test.mock import FakeLima


backend= aer_simulator(fake_lima) 111


In [2]:
pm_config = PassManagerConfig(initial_layout=None,
        basis_gates=backend.configuration().basis_gates,
        inst_map=None,
        coupling_map=CouplingMap(backend.configuration().coupling_map),
        layout_method=None,
        routing_method=None,
        translation_method=None,
        scheduling_method=None,
        instruction_durations=None,
        backend_properties=backend.properties(),
        approximation_degree=None,
        seed_transpiler=seed,
        timing_constraints=None,
        unitary_synthesis_method="default",
        unitary_synthesis_plugin_config=None,
        target=None)

In [18]:
pm = level_3_pass_manager(pm_config)
pprint(pm.passes())

[{'flow_controllers': {},
  'passes': [<qiskit.transpiler.passes.synthesis.unitary_synthesis.UnitarySynthesis object at 0x7efea597d4c0>,
             <qiskit.transpiler.passes.basis.unroll_3q_or_more.Unroll3qOrMore object at 0x7efea597d850>]},
 {'flow_controllers': {},
  'passes': [<qiskit.transpiler.passes.optimization.remove_reset_in_zero_state.RemoveResetInZeroState object at 0x7efea46a8f40>,
             <qiskit.transpiler.passes.optimization.remove_diagonal_gates_before_measure.RemoveDiagonalGatesBeforeMeasure object at 0x7efea46a09d0>]},
 {'flow_controllers': {},
  'passes': [<qiskit.transpiler.passes.layout.set_layout.SetLayout object at 0x7efea597da00>]},
 {'flow_controllers': {'condition'},
  'passes': [<qiskit.transpiler.passes.layout.vf2_layout.VF2Layout object at 0x7efea597db50>]},
 {'flow_controllers': {'condition'},
  'passes': [<qiskit.transpiler.passes.layout.sabre_layout.SabreLayout object at 0x7efea597dbb0>]},
 {'flow_controllers': {},
  'passes': [<qiskit.transpiler.

In [19]:
# delete the Optimize1qGatesDecomposition pass - IMPORTANT to avoid gate merging
del pm.passes()[-1]['passes'][3]

In [20]:
qc1=qk.QuantumCircuit.from_qasm_file('circ28.qasm')
print("-------------------TRANSPILE 2----------------------------------")
qc1t=pm.run(qc1)
print(qc1)

qc2=qk.QuantumCircuit.from_qasm_file('circ29.qasm')
print("-------------------TRANSPILE 2----------------------------------")
qc2t=pm.run(qc2)
print(qc2)

#  compare results
sum1=str(qc1t.count_ops())
sum2=str(qc2t.count_ops())

print('qc1 transp depth=',qc1t.depth(),sum1)
print('qc2 transp depth=',qc2t.depth(),sum2)
assert qc1t.depth()==qc2t.depth()  #test
assert sum1==sum2 # test 2 count gates and all other details
print('M:ok')

-------------------TRANSPILE 2----------------------------------
       ┌────────────┐┌───┐┌──────────────┐┌───┐┌───────────┐┌───┐┌───────────┐»
  q_0: ┤ Ry(1.6035) ├┤ X ├┤ Ry(-0.35997) ├┤ X ├┤ Ry(-π/96) ├┤ X ├┤ Ry(-π/96) ├»
       └───┬───┬────┘└─┬─┘└──────────────┘└─┬─┘└───────────┘└─┬─┘└───────────┘»
  q_1: ────┤ H ├───────■────────────────────┼─────────────────■───────────────»
           ├───┤                            │                                 »
  q_2: ────┤ H ├────────────────────────────■─────────────────────────────────»
           ├───┤                                                              »
  q_3: ────┤ H ├──────────────────────────────────────────────────────────────»
           └───┘                                                              »
c28: 4/═══════════════════════════════════════════════════════════════════════»
                                                                              »
«       ┌───┐┌─────────────┐┌───┐┌──────────┐┌───┐┌────